In [ ]:
import time
import warnings                           # to ignore runtime warnings
import numpy as np
import pandas as pd                       # pandas for data preprocessing
import seaborn as sns                     # seaborn for data visualization
import cufflinks as cf                    # cufflinks for plotly 
import chart_studio.plotly as py
import plotly.express as px               # plotly for advance data visualization
import speech_recognition as sr           # speech_recognition library for taking voice as an input
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs , plot , iplot , init_notebook_mode
init_notebook_mode(connected = True)
cf.go_offline()
warnings.filterwarnings('ignore')

In [ ]:
high_thresh_freq = 0
high_thresh_word = str
total_words = 0
words_repeated = 0
filler_words=0
question_no=1
total_pacing=0
total_polish=0
total_power=0
''' we start by creating a parent class recurrent phrase
                            and declare the filler words in the constructor of parent class '''


class Recurrent_phrase:     # parent class
    
    def __init__(self):     # parent class constructor
        
        # declaring the filler words
        self.words_to_recognize = [
                        "well",'um','uh',"hmm","like","actually","basically","seriously","you see","you know","I mean",
                         "at the end of the day" , "believe me", "I guess" , "I suppose" , "or something" , "ok so" , "right"
                      ]
        
        # creating instance for speech recognition
        self.speech_1 = sr.Recognizer() 


        
''' create a class frequency_check to check the frequency of the filler words if they occur in the sentence '''

class frequency_check(Recurrent_phrase):      # child class frequency_check inherits parent class Recurrent_phrase
    
    def __init__(self):                       # child class constructor
        
        # calling the constructor of parent class
        super().__init__()
        
        
        
        
    ''' we now create a function to check whether the filler words occur in the sentence '''
    
    def character_check(self):                           # defining a function character_check
        self.words = []                                  # creating the list to append the words if they appear
        with sr.Microphone() as source:                  # using Microphone function to take input from the microphone
            start_time = time.time()
            print('\nRespond to the above question')
            audio = self.speech_1.listen(source)         # using listen function to listen the source
            end_time = time.time()
            self.character = self.speech_1.recognize_google(audio) # using google's API to recognize the words
            for char in self.words_to_recognize:
                if char in self.character:
                    self.words.append(char)              # appending the filler words in the list if they exists in the sentence
                    
            self.total_time = end_time - start_time
            
                    
    ''' we now create a function to check the frequency of those filler words '''
    
    def check_frequency(self):                       # defining a function check_frequency
        self.character_check()
        self.frequency = {}                          # creating a dictionary to store the filler words if they exist
        for i in self.words:
            x = self.character.count(i)
            if i not in self.frequency:
                self.frequency[i] = x               
            else:
                self.frequency[i] += 1              
        return self.character , self.total_time/60      # returning the input given by the user
    
    
    
    
''' visualizing the frequency of filler words , here we are using python plotly library for advance data visualization '''

class visualize_frequency(frequency_check):         # child class visualize_frequency inherits the parent chass frequency_check 
    
    def __init__(self):                             # constructor of child class 
        
        # calling the parent class constructor
        super().__init__()
       
    
    # creating a dataset of the filler words
    def data_set(self):
         self.data = pd.DataFrame(pd.Series(self.frequency) , columns = ['Frequency'])
    
    # calculating frequency of filler words
    def time_and_frequency(self):
        self.data_set()
        global total_words
        global filler_words
        total_words=0
        filler_words=0
        total_words=len(self.character.split())
        if len(self.frequency) != 0:
            print('\nFrequency of filler words in your speech :')
            for i in self.frequency:
                print(i ,':', self.frequency[i]/total_words*100 , end = '  ')
                filler_words = filler_words + self.frequency[i]
            else:
                pass
        else:
            pass
        
    def words_per_minute(self):
        self.time_and_frequency()
        self.wps = len(self.character.split())/(self.total_time/60)
        
    # visualizing the dataset
    def visualize(self):
        self.words_per_minute()
        
        if len(self.data) != 0:                     # checking whether we have encountered any filler word or not
            
            # creating a bar plot of the frequencies of the filler words using bar plt
            fig = px.bar(data_frame = self.data , x = self.data.index , y = 'Frequency',color = self.data.index)
            fig.show()
            
        else:
            pass
        return self.wps
    
    def total_count_of_words(self):
        words_by_user = {}
        thresh = 2         #setting a min value for words repeated
        global high_thresh_freq
        global high_thresh_word
        global words_repeated
        high_thresh_freq=0
        high_thresh_word=None
        words_repeated=0
        
        for words in self.character.split():
            if words not in words_by_user:
                words_by_user[words] = 1
            else:
                words_by_user[words] += 1
                
        for key , value in words_by_user.items():
            if value > high_thresh_freq:
                high_thresh_word = key
                high_thresh_freq = value
            if value > thresh:
                words_repeated = words_repeated + value
                yield key
            else:
                pass
        return words_repeated
            
'''Questions for each prompt'''
science={1:"What are some ways you can play a role in mitigating climate change?", 
         2:"Has technology had a net positive or negative impact on life and why?",
        3:"What is one statistic that you find super important and why?",
        4:"Do you believe society makes choices based on data or emotion? What is more important?",
        5:"What are the values of learning computer science in school and how can schools better teach it?",
        6:"Why is it important to be able to understand and communicate data to people, and how can you hone your skills in doing this?",
        7:"What are some ways you can take care of your body and mind to be healthier and more productive in school and extracurricular wise?",
        8:"What is one way you can use technology to create a new product or idea that would add value to your life or someone else's life?"}

arts={1:"What is a piece of art or literature that had a strong impact on you and what was this impact?", 
         2:"Do you believe you are a creative person and why is creativity important? What are some ways you can become more creative?",
        3:"What are some of the values and advantages to teaching the arts and humanities in education?",
        4:'It is sometimes claimed that "Art is something which generates an emotional reaction." What is your opinion?',
        5:"What was one piece of art that you created, a poem you wrote, or an essay that you worked hard on and means a lot to you? What was the process for creating it and what did you gain from that experience?",
        6:"Does art always have to send a message to the person viewing it? If not, what is the purpose of art?",
        7:"What are some ways we can combine arts and science to make a difference in the world?",
        8:"How will you use the arts and literature in your future path and career?"}

professional={1:"What is one project you have worked on in school or outside of school and what skills did you gain from that experience?", 
         2:"What are your biggest strengths, and how do you plan to use it in your life and career?",
        3:"Who are some of your role models and mentors, and what have you learned from them?",
        4:"What type of impact do you want to make in the world and how do you plan to make this impact?",
        5:"In the future, what is one career destination you want to arrive at (a dream company, a dream university, etc.). Why is it a destination, and how can you get there?",
        6:"What are some of your biggest improvement areas and how can you work on these weaknesses to become better?",
        7:"What are some ways you prioritize your mental health while staying productive?",
        8:"Have you worked on a team before, and what are some of your strengths and weaknesses as a team member?"}

logic={1:"Is social media a net positive or negative for society?", 
         2:"Is it every individual's duty to participate in politics or not?",
        3:"Is online or in-person learning better?",
        4:"Is learning the arts of technology more important to our educational growth?",
        5:"What is the biggest problem facing the world today?",
        6:"Is a college education necessary for success or not?",
        7:"What is the ideal form of government that will maximise the health and happiness of society?",
        8:"How would you define happiness, and is it necessary to have money to be happy?"}

if __name__ == '__main__':
    
    def reset():
        global total_pacing
        global total_polish
        global total_power
        total_pacing=0
        total_polish=0
        total_power=0
    
    def threshold():         #function defining criteria for evaluation
        global high_thresh_freq
        global high_thresh_word
        global total_words
        global words_repeated
        global filler_words
        global total_pacing
        global total_polish
        global total_power
        vi = visualize_frequency()
        word_spoken , total_time_taken = vi.check_frequency()
        words_per_minute = vi.visualize()
        words=list(vi.total_count_of_words())
        
        #unique words spoken
        power = ((total_words - words_repeated)/total_words)*100
        print('\nsentence spoken by the user : ',word_spoken)
        print('\ntotal time taken by the user : ',np.round(total_time_taken,2),'mins')
        print('\ntotal words per minute by the user : ',np.round(words_per_minute,2))
        
        # polish threshold
        if(filler_words < 30):
            print("\nPolish:",filler_words,",  You used little to no filler words! Continue to think about your sentences before you say them to avoid having to use filler words when you're not sure what to say")
        elif(30 < filler_words < 60):
            print("\nPolish:",filler_words,",  You used a few more filler words than the target. Specifically, you said these filler words: Try to think actively about not using filler words, and when you feel one coming, don't be afraid to pause during the sentence")
        else:
            print("\nPolish:",filler_words,",  You used a significant number of filler words. Specifically, you said these filler words:Specifically, you said these filler words: You will not be able to eliminate them all in one go, but try finding one word you use and focus on not using it. You will eventually get there!!")
        time.sleep(5)
        
        # pacing threshold
        if(words_per_minute < 100):
            print("\nPacing:",words_per_minute,", You spoke ",100 - words_per_minute, " words per minute lower than the objective. Take a few seconds before you answer the prompt to organize your thoughts and practice what you are going to say in your head")
        elif(100 < words_per_minute < 200):
            print("\nPacing:",words_per_minute,", Your pacing was within the target! Keep up the good work :)")
        else:
            print("\nPacing:",words_per_minute,", You spoke ",words_per_minute - 200," words per minute faster than the target. Make sure you are pronouncing every word completely, and really thinking through each question. You should be actively thinking during your pauses so you can deliver the best sentences ever!!")
        time.sleep(5)
         
        #power threshold
        if(power > 30):
            print("\nPower:",power, ", You did not repeat many words and you were very deliberate and powerful with your speech. In a movie, if they repeated the same scene many times, it would lose its power and you would not enjoy the movie. Treat your words and phrases like this, and keep your response linear and progressing.")
        else:
            print("\nPower:",power, ", While you were under the threshold, you came quite close to being too repetitive. Specifically, you used the words: " "",high_thresh_word,"" " more than ",high_thresh_freq," times. Make sure to use repetition deliberately. Some words and phrases are powerful when repeated, but if you are just repeating words or phrases because you have nothing new to say, you will not sound powerful")
        time.sleep(5)
        
        print("\nLet's move on!")
        total_pacing=total_pacing+words_per_minute
        total_polish=total_polish+filler_words
        total_power=total_power+power
        return 0
        
    def prompts(prompt):    
        global question_no
        
        if prompt == 'science and technology':
            print("\nQuestion",question_no, "-",science[question_no])
            val=threshold()
            if val==0:                        #appending through the questions of the prompt
                question_no=question_no+1
            if question_no<9:
                prompts(prompt)
            
        elif prompt == 'arts and humanities':
            print("\nQuestion",question_no, "-",arts[question_no])
            val=threshold()
            if val==0:
                question_no=question_no+1
            if question_no<9:
                prompts(prompt)
            
        elif prompt == 'professional development':
            print("\nQuestion",question_no, "-",professional[question_no])
            val=threshold()
            if val==0:
                question_no=question_no+1
            if question_no<9:
                prompts(prompt)
            
        elif prompt == 'logic and debate':
            print("\nQuestion",question_no, "-",logic[question_no])
            val=threshold()
            if val==0:
                question_no=question_no+1
            if question_no<9:
                prompts(prompt)
            
    chance = 1
    print('Hello and Welcome to Valfee Feedback Game!!\n')
    prompt = input('Choose one of the following categories: science and technology, arts and humanities, professional development, logic and debate : ')
    while (chance < 3):
        reset()
        prompts(prompt)
        if ((total_polish/8) < 30 and (100 < (total_pacing/8) < 200) and (total_power/8) > 30):
            print('\nCongratulations! You have successfully completed this game with average scores -')
            print('Your Average Pacing - ',total_pacing/8,'words per minute.')
            print('Your Average Polish - ',total_polish/8,'filler words.')
            print('Your Average Power - ',total_power/8)
            break
        else:
            print('\nYour speech was not to the point, please try to improve it and try again')
            question_no = 1
            chance = chance + 1
        
        
